In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!unzip drive/My\ Drive/bluebook-for-bulldozers

Archive:  drive/My Drive/bluebook-for-bulldozers.zip
  inflating: TrainAndValid.csv       
  inflating: Train.zip               
  inflating: Train.7z                
  inflating: Valid.7z                
  inflating: Test.csv                
  inflating: Machine_Appendix.csv    
  inflating: median_benchmark.csv    
  inflating: ValidSolution.csv       
  inflating: TrainAndValid.7z        
  inflating: random_forest_benchmark_test.csv  
  inflating: Valid.zip               
  inflating: Data Dictionary.xlsx    
  inflating: TrainAndValid.zip       
  inflating: Valid.csv               


In [3]:
!unzip Train.zip

Archive:  Train.zip
  inflating: Train.csv               


### 1. Dependencies

In [0]:
# !pip install fastai==0.7.0

In [0]:
from fastai.imports import *
from fastai.structured import *

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

## 2. Data Pipeline

### 2.1 Loading the data from the directory

In [0]:
train_df = pd.read_csv('TrainAndValid.csv', low_memory=False,  parse_dates=["saledate"])
test_df = pd.read_csv('Test.csv', low_memory=False,  parse_dates=["saledate"])
# val_df = pd.read_csv('Valid.csv', low_memory=False,  parse_dates=["saledate"])

In [47]:
print(train_df.shape)
print(test_df.shape)
#print(val_df.shape)

(412698, 53)
(12457, 52)


### 2.2 Viewing the insights of the dat

In [14]:
train_df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
SalesID                     412698 non-null int64
SalePrice                   412698 non-null float64
MachineID                   412698 non-null int64
ModelID                     412698 non-null int64
datasource                  412698 non-null int64
auctioneerID                392562 non-null float64
YearMade                    412698 non-null int64
MachineHoursCurrentMeter    147504 non-null float64
UsageBand                   73670 non-null object
saledate                    412698 non-null datetime64[ns]
fiModelDesc                 412698 non-null object
fiBaseModel                 412698 non-null object
fiSecondaryDesc             271971 non-null object
fiModelSeries               58667 non-null object
fiModelDescriptor           74816 non-null object
ProductSize                 196093 non-null object
fiProductClassDesc          412698 non-null object
state              

In [16]:
train_df.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,412698,NaN,NaN,NaN,NaN,NaN,2.01116e+06,1.08007e+06,1.13925e+06,1.4219e+06,1.64585e+06,2.26101e+06,6.33335e+06
SalePrice,412698,NaN,NaN,NaN,NaN,NaN,31215.2,23141.7,4750,14500,24000,40000,142000
MachineID,412698,NaN,NaN,NaN,NaN,NaN,1.23006e+06,453953,0,1.08859e+06,1.2844e+06,1.47808e+06,2.48633e+06
ModelID,412698,NaN,NaN,NaN,NaN,NaN,6947.2,6280.82,28,3261,4605,8899,37198
datasource,412698,NaN,NaN,NaN,NaN,NaN,135.169,9.64675,121,132,132,136,173
auctioneerID,392562,NaN,NaN,NaN,NaN,NaN,6.58527,17.1584,0,1,2,4,99
YearMade,412698,NaN,NaN,NaN,NaN,NaN,1899.05,292.19,1000,1985,1995,2001,2014
MachineHoursCurrentMeter,147504,NaN,NaN,NaN,NaN,NaN,3522.99,27169.9,0,0,0,3209,2.4833e+06
UsageBand,73670,3,Medium,35832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,412698,4013,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2012-04-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
train_df.SalePrice.head()

0    66000.0
1    57000.0
2    10000.0
3    38500.0
4    11000.0
Name: SalePrice, dtype: float64

In [0]:
# To calculate RMSLE (root mean squared log error) between actual and predicted price.
# The evaluation metric for this competition is the RMSLE (root mean squared log error)
# between the actual and predicted auction prices.
train_df.SalePrice = np.log(train_df.SalePrice)

In [0]:
price = train_df.SalePrice

In [0]:
# drop the SalePrice from the train_df and process the train_df and test_df toegther
train_df = train_df.drop('SalePrice', axis=1)

In [65]:
print(train_df.shape, test_df.shape)

(412698, 52) (12457, 52)


In [72]:
if list(train_df.columns) == list(test_df.columns):
  print('both DataFrame are same')
else:
  print('DataFrames are different')

both DataFrame are same


In [0]:
df = train_df.append(test_df)

In [74]:
df.shape

(425155, 52)

In [70]:
type(list(df.columns))

list

In [0]:
# dividing the date into multiple subfeatures so that we can extract more information from it
add_datepart(df, 'saledate')

In [76]:
df.shape

(425155, 64)

### 2.3 Converting the categorical data into numerical values

In [0]:
# seperate the numerical values from categorical values
all_features = df.columns
numerical = df.select_dtypes(include=['number']).copy()
categorical = df.select_dtypes(include=['category']).copy()

In [90]:
print(numerical.shape)
print(categorical.shape)

(425155, 14)
(425155, 44)


In [0]:
# convert the string values into pandas categorical values
train_cats(df)

In [81]:
print(df.state.cat.categories)

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Unspecified', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'Washington DC', 'West Virginia', 'Wisconsin',
       'Wyoming'],
      dtype='object')


In [83]:
# see the null values
display_all(df.isnull().sum())

SalesID                          0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    275522
UsageBand                   349651
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             144702
fiModelSeries               364482
fiModelDescriptor           347315
ProductSize                 223014
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                315309
Enclosure                      336
Forks                       221132
Pad_Type                    341951
Ride_Control                268186
Stick                       341951
Transmission                232330
Turbocharged                341951
Blade_Extension             398521
Blade_Width                 398521
Enclosure_Type      

In [0]:
df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [0]:
df.UsageBand = df.UsageBand.cat.codes

In [97]:
print(train_df.shape, test_df.shape)

(412698, 52) (12457, 52)


In [0]:
train, test = df[:train_df.shape[0]], df[train_df.shape[0]:]

In [109]:
print(train.shape, test.shape)

(412698, 64) (12457, 64)


In [110]:
train['SalePrice'] = price

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [112]:
#Replace categories with numerical codes
x, y, nas = proc_df(train, 'SalePrice')


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## 3. Model

In [0]:
rf = RandomForestRegressor(n_jobs=-1)

In [115]:
rf.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [116]:
rf.score(x,y)

0.9826547826587448

## make prediction

In [128]:
x_test, y_test, nas = proc_df(test)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
x_test['auctioneerID_na'] = False

In [0]:
pred = rf.predict(x_test)

In [0]:
result = pd.DataFrame({'SalesID': x_test.SalesID, 'SalePrice': pred})

In [0]:
result.to_csv('result.csv', index=False)